In [2]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [11]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import time

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('Fortified')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased
            time.sleep(10)
            wait(driver, 20).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [45]:
from wine import Wine
port_wine_url = 'https://www.vivino.com/explore?e=eJwlizsOgzAQBW_zyohv-TpuEKVCEdoYY1mKIdo1JLk9lmhGM8UkZXvrkeLKCkl-bPoK7s_HHa5gwIc1wsJDNPosb2wvquS4Bpvk8CrBY-PszeGbxyebrtxWZDfo1UYnJ7-wIhE='
port_wine = Wine(port_wine_url)
port_wine.setup_driver()
port_wine.run_timestamp()
lazyloading(port_wine.driver)
port_wine.run_timestamp()

Current Timestamp:  01-Feb-2020 (20:35:06.103581)
Total Number:  194
Curr Row Number:  50
Curr Row Number:  75
Curr Row Number:  100
Curr Row Number:  125
Curr Row Number:  150
Curr Row Number:  175
Curr Row Number:  194
Current Timestamp:  01-Feb-2020 (20:36:17.566813)


In [40]:
port_html = BeautifulSoup(port_wine.driver.page_source, 'lxml')
port_div = port_html.find("div", {"class": "explorerPage__results--3wqLw"})
port_rows = port_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [41]:
len(port_rows)

1060

In [42]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "region": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "image_url": None
}

for row in port_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Port'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['review'] = review_div.text
    image_div = row.find("div", {"class": "cleanWineCard__bottleShotWrapper--nymTj"})
    if image_div:
        new_row['image_url'] = image_div.find("div").find_next('img')['src'].strip("//")
    all_rows.append(new_row)

In [43]:
all_rows

[{'title': 'Quinta do Seixo Vintage Porto 2017',
  'location': 'Vintage Port',
  'region': None,
  'country': 'Portugal',
  'price': 61.31,
  'type': 'Port',
  'ratings': '4.8',
  'num_ratings': 97,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/y9P75uaVT6yoGz2398QDDw_pb_x300.png'},
 {'title': 'Vintage Port 1994',
  'location': 'Vintage Port',
  'region': None,
  'country': 'Portugal',
  'price': 227.42,
  'type': 'Port',
  'ratings': '4.7',
  'num_ratings': 325,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/y9P75uaVT6yoGz2398QDDw_pb_x300.png'},
 {'title': 'Very Old Single Harvest Port 1968',
  'location': 'Tawny Port',
  'region': None,
  'country': 'Portugal',
  'price': 247.19,
  'type': 'Port',
  'ratings': '4.7',
  'num_ratings': 165,
  'reviews': None,
  'image_url': 'images.vivino.com/thumbs/y9P75uaVT6yoGz2398QDDw_pb_x300.png'},
 {'title': 'Vintage Port 1994',
  'location': 'Vintage Port',
  'region': None,
  'country': 'Portugal',
  'price': 222.47,
 

In [17]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [22]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine']
types = wine_db['types']

In [44]:
for row in all_rows:
    types.insert_one(row)